## Geometric Manipulations

##### geopandas makes available all the tools for geometric manipulations in the *shapely* library.


* **Constructive Methods**

    * **GeoSeries.buffer(distance, resolution=16)**: 
    Returns a GeoSeries of geometries representing all points within a given distance of each geometric object.

    * **GeoSeries.boundary**
    Returns a GeoSeries of lower dimensional objects representing each geometries’s set-theoretic boundary.

    * **GeoSeries.centroid**
    Returns a GeoSeries of points for each geometric centroid.

    * **GeoSeries.convex_hull**
    Returns a GeoSeries of geometries representing the smallest convex Polygon containing all the points in each object unless the number of points in the object is less than three. For two points, the convex hull collapses to a LineString; for 1, a Point.
    * **GeoSeries.envelope**
    Returns a GeoSeries of geometries representing the point or smallest rectangular polygon (with sides parallel to the coordinate axes) that contains each object.

    GeoSeries.simplify(tolerance, preserve_topology=True)
    Returns a GeoSeries containing a simplified representation of each object.

    * **GeoSeries.unary_union**
    Return a geometry containing the union of all geometries in the GeoSeries.




* **Affine transformations**

    * **GeoSeries.rotate(self, angle, origin='center', use_radians=False)**
    Rotate the coordinates of the GeoSeries.

    * **GeoSeries.scale(self, xfact=1.0, yfact=1.0, zfact=1.0, origin='center')**
    Scale the geometries of the GeoSeries along each (x, y, z) dimensio.

    * **GeoSeries.skew(self, angle, origin='center', use_radians=False)**
    Shear/Skew the geometries of the GeoSeries by angles along x and y dimensions.

    * **GeoSeries.translate(self, angle, origin='center', use_radians=False)**
    Shift the coordinates of the GeoSeries.

## Example

### **Data:** http://www1.nyc.gov/site/planning/data-maps/open-data/districts-download-metadata.page

In [ ]:
import geopandas as gpd
import numpy as np
%matplotlib inline

In [ ]:
boros = gpd.GeoDataFrame.from_file(r'.\DATA\nybb.shp')
boros.set_index('BoroCode', inplace=True)
boros.sort()
boros

In [ ]:
boros.plot(figsize=(12, 6))

In [ ]:
boros['geometry'].convex_hull.plot(figsize=(12, 6))


In [ ]:
#generate a GeoSeries containing 200 random points:

from shapely.geometry import Point
xmin, xmax, ymin, ymax = 900000, 1080000, 120000, 280000
xc = (xmax - xmin) * np.random.random(200) + xmin
yc = (ymax - ymin) * np.random.random(200) + ymin
pts = gpd.GeoSeries([Point(x, y) for x, y in zip(xc, yc)])
pts

In [ ]:
#Now draw a circle with fixed radius around each point:

circles = pts.buffer(4000)

circles.plot(figsize=(12, 6))

In [ ]:
#We can collapse these circles into a single shapely MultiPolygon geometry with

mp = circles.unary_union
mp

In [ ]:
#To extract the part of this geometry contained in each borough, we can just use:

holes = boros['geometry'].intersection(mp)
holes.plot(figsize=(12, 6))

## Overlay Set Operations

![alt text](Picture1.png "Title")

## Overlay Countries Example
First, we load the countries and cities example datasets and select :

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

capitals = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

# Select South Amarica and some columns
countries = world[world['continent'] == "South America"]

countries = countries[['geometry', 'name']]

# Project to crs that uses meters as distance measure
countries = countries.to_crs('+init=epsg:3395')

capitals = capitals.to_crs('+init=epsg:3395')
# To illustrate the overlay function, consider the following case in which one 
# wishes to identify the “core” portion of each country – defined as areas within 500km of a 
# capital – using a GeoDataFrame of countries and a GeoDataFrame of capitals.

# Look at countries:
countries.plot(figsize=(12, 6));

In [ ]:
# Now buffer cities to find area within 500km.
# Check CRS -- World Mercator, units of meters.
capitals.crs


In [ ]:
# make 500km buffer
capitals['geometry']= capitals.buffer(500000)

capitals.plot(figsize=(12, 6))

### To select only the portion of countries within 500km of a capital, we specify the how option to be “intersect”, which creates a new set of polygons where these two layers overlap:

In [ ]:
country_cores = gpd.overlay(countries, capitals, how='intersection')

In [ ]:
country_cores

In [ ]:
 country_cores.plot(figsize=(12, 6))

#### Changing the “how” option allows for different types of overlay operations. 
For example, if we were interested in the portions of countries far from capitals (the peripheries), we would compute the difference of the two.


In [ ]:

country_peripheries = gpd.overlay(countries, capitals, how='difference')

In [ ]:
country_peripheries.plot(figsize=(12, 6))

# Aggregation with dissolve

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world = world[['continent', 'geometry']]

continents = world.dissolve(by='continent')

continents.plot(figsize=(12, 6))

continents.head()

If we are interested in aggregate populations, however, we can pass different functions to the dissolve method to aggregate populations:



In [ ]:
import PySAL
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world = world[['continent', 'geometry', 'pop_est']]

continents = world.dissolve(by='continent', aggfunc='sum')

continents.plot(column = 'pop_est', scheme='quantiles', cmap='YlOrRd');

continents.sort_values(by='pop_est').head(10)

## Merging Data

There are two ways to combine datasets in geopandas – attribute joins and spatial joins.

In an attribute join, a GeoSeries or GeoDataFrame is combined with a regular pandas Series or DataFrame based on a common variable. This is analogous to normal merging or joining in pandas.

In a Spatial Join, observations from to GeoSeries or GeoDataFrames are combined based on their spatial relationship to one another.



In [ ]:
#In the following examples, we use these datasets:

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

# For attribute join
country_shapes = world[['geometry', 'iso_a3']]

country_names = world[['name', 'iso_a3']]


In [ ]:
# Attribute Joins

country_shapes.head()

In [ ]:
country_names.head()


In [ ]:
country_shapes = country_shapes.merge(country_names, on='iso_a3')
country_shapes.head()

In [ ]:
#Spatial Joins
#In a Spatial Join, two geometry objects are merged based on their spatial relationship to one another.


# For spatial join
countries = world[['geometry', 'name']]

countries = countries.rename(columns={'name':'country'})

# One GeoDataFrame of countries, one of Cities.
# Want to merge so we can get each city's country.
countries.head()


In [ ]:
cities.head()


In [ ]:
cities_with_country = gpd.sjoin(cities, countries, how="inner", op='intersects')

cities_with_country.head()